<a href="https://colab.research.google.com/github/thanhtrung102/FOREX-01/blob/main/MA_Cross_Overforecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the libraries we need

In [ ]:
!pip install pandas_ta
!pip install Meta
import pandas as pd
import pandas_ta as ta
import numpy as np
import MetaTrader5 as mt5
from   datetime import datetime
import seaborn as sns
import time

ModuleNotFoundError: No module named 'pandas_ta'

Setup log in details

In [ ]:
account = 30516497
password = "4ATSm7pPxK4iuW5*"
server = "Deriv-Demo"

In [ ]:
if(mt5.initialize(login=account,password=password,server=server)):
    print("Logged in succesfully")
else:
    print("Failed to login")

Market data

In [ ]:
timeframe = mt5.TIMEFRAME_M15
deviation = 1000
volume = 0
lot_multiple = 10
symbol = "EURUSD"

In [ ]:
#Setup trading volume
symbols = mt5.symbols_get()
for index,symbol in enumerate(symbols):
    if symbol.name == "EURUSD":
        print(f"{symbol.name} has minimum volume: {symbol.volume_min}")
        volume = symbol.volume_min * lot_multiple

Get Training Data

In [ ]:
#Specify date range of data to be modelled
date_start = datetime(2020,1,1)
date_end = datetime.now()

In [ ]:
#Define how far ahead we are looking
look_ahead = 20

In [ ]:
#Fetch market data
market_data = pd.DataFrame(mt5.copy_rates_range("EURUSD",timeframe,date_start,date_end))
market_data["time"] = pd.to_datetime(market_data["time"],unit='s')
#Add simple moving average technical indicator
market_data.ta.sma(length=5,append=True)
#Add simple moving average technical indicator
market_data.ta.sma(length=50,append=True)
#Delete missing rows
market_data.dropna(inplace=True)
#Add a column for the target
market_data["target"] = 0
market_data["close_target"] = 0
#Encoding the target
ma_cross_conditions = [
    (market_data["SMA_5"].shift(-look_ahead) > market_data["SMA_50"].shift(-look_ahead)),
    (market_data["SMA_5"].shift(-look_ahead) < market_data["SMA_50"].shift(-look_ahead))
]
#Encoding pattern
ma_cross_choices = [
    #Fast MA above Slow MA
    1,
    #Fast MA below Slow MA
    0
]

price_conditions = [
    (market_data["close"] > market_data["close"].shift(-look_ahead)),
    (market_data["close"] < market_data["close"].shift(-look_ahead))
]

#Encoding pattern
price_choices = [
    #Price fell
    0,
    #Price rose
    1
]

market_data["target"] = np.select(ma_cross_conditions,ma_cross_choices)
market_data["close_target"] = np.select(price_conditions,price_choices)

#The last rows do not have answers
market_data = market_data[:-look_ahead]
market_data

Importing Libraries

In [ ]:
#XGBoost
from xgboost import XGBClassifier
#Catboost
from catboost import CatBoostClassifier
#Random forest
from sklearn.ensemble import RandomForestClassifier
#LDA and QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis , QuadraticDiscriminantAnalysis
#Logistic regression
from sklearn.linear_model import LogisticRegression
#Neural network
from sklearn.neural_network import MLPClassifier
#Time series split
from sklearn.model_selection import TimeSeriesSplit
#Accuracy metrics
from sklearn.metrics import accuracy_score
#Visualising performance
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

Setting up Time Series Split

In [ ]:
#Time series split
splits = 10
gap = look_ahead

In [ ]:
models = ["Logistic Regression","Linear Discriminant Analysis","Quadratic Discriminant Analysis","Random Forest Classifier","XGB Classifier","Cat Boost Classifier","Neural Network Small","Neural Network Large"]

Data Frame To Store Model Performance

In [ ]:
error_ma_crossover = pd.DataFrame(index=np.arange(0,splits),columns=models)
error_ma_crossover

In order to have a benchmark to compare our performance we will repeat the same test with the actuall change in original price as the target

In [ ]:
error_price = pd.DataFrame(index=np.arange(0,splits),columns=models)
error_price

Proceeding With Model Cross-Validation

In [ ]:
predictors = ["open","high","low","close","tick_volume","spread","SMA_5","SMA_50"]

In [ ]:
tscv = TimeSeriesSplit(n_splits=splits,gap=gap)

In [ ]:
#Training each model to predict changes in the moving average cross over
for i,(train,test) in enumerate(tscv.split(market_data)):
    model = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(20, 10), random_state=1,early_stopping=True)
    model.fit( market_data.loc[train[0]:train[-1],predictors] , market_data.loc[train[0]:train[-1],"target"] )
    error_ma_crossover.iloc[i,7] = accuracy_score(market_data.loc[test[0]:test[-1],"target"],model.predict(market_data.loc[test[0]:test[-1],predictors]))

In [ ]:
#Training each model to predict changes in the close price
for i,(train,test) in enumerate(tscv.split(market_data)):
    model = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(20, 10), random_state=1,early_stopping=True)
    model.fit( market_data.loc[train[0]:train[-1],predictors] , market_data.loc[train[0]:train[-1],"close_target"] )
    error_price.iloc[i,7] = accuracy_score(market_data.loc[test[0]:test[-1],"close_target"],model.predict(market_data.loc[test[0]:test[-1],predictors]))

Results

In [ ]:
error_ma_crossover

In [ ]:
error_price

Visualising Results

Box Plots

In [ ]:
fig,axs = plt.subplots(1,8,sharex=True,sharey=True,figsize=(30,10))

for i,ax in enumerate(axs.flat):
    ax.boxplot(error_ma_crossover.iloc[:,i])
    ax.set_title(models[i])

In [ ]:
fig,axs = plt.subplots(1,8,sharex=True,sharey=True,figsize=(30,10))

for i,ax in enumerate(axs.flat):
    ax.boxplot(error_price.iloc[:,i])
    ax.set_title(models[i])

Confusion Matrices

Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(market_data.loc[49:1000,predictors],market_data.loc[49:1000,"target"])
cm = confusion_matrix(market_data.loc[1030:,"target"], model.predict(market_data.loc[1030:,predictors]))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix For Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('True')

Linear Discriminant Analysis

In [ ]:
model = LinearDiscriminantAnalysis()
model.fit(market_data.loc[49:1000,predictors],market_data.loc[49:1000,"target"])
cm = confusion_matrix(market_data.loc[1030:,"target"], model.predict(market_data.loc[1030:,predictors]))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix For Linear Discrimininant Analysis')
plt.xlabel('Predicted')
plt.ylabel('True')

XGB Classifier

In [ ]:
model = XGBClassifier()
model.fit(market_data.loc[49:1000,predictors],market_data.loc[49:1000,"target"])
cm = confusion_matrix(market_data.loc[1030:,"target"], model.predict(market_data.loc[1030:,predictors]))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix For XGBoost Classifier')
plt.xlabel('Predicted')
plt.ylabel('True')

Small Neural Network

In [ ]:
model = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(4, 2), random_state=1)
model.fit(market_data.loc[49:1000,predictors],market_data.loc[49:1000,"target"])
cm = confusion_matrix(market_data.loc[1030:,"target"], model.predict(market_data.loc[1030:,predictors]))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix For Small Neural Network')
plt.xlabel('Predicted')
plt.ylabel('True')

Learning Curve

In [ ]:
model = LogisticRegression()
train_sizes, train_scores, test_scores = learning_curve(model, market_data.loc[:,predictors],market_data.loc[:,"target"], cv=10)

plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve For Logistic Regression')
plt.legend()
plt.show()

Linear Discriminant Analysis

In [ ]:
model = LinearDiscriminantAnalysis()
train_sizes, train_scores, test_scores = learning_curve(model, market_data.loc[:,predictors],market_data.loc[:,"target"], cv=10)

plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve For Linear Discriminant Analysis')
plt.legend()
plt.show()

In [ ]:
model = QuadraticDiscriminantAnalysis()
train_sizes, train_scores, test_scores = learning_curve(model, market_data.loc[:,predictors],market_data.loc[:,"target"], cv=10)

plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve For Quadratic Discriminant Analysis')
plt.legend()
plt.show()

In [ ]:
model =CatBoostClassifier()
train_sizes, train_scores, test_scores = learning_curve(model, market_data.loc[:,predictors],market_data.loc[:,"target"], cv=10)

plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve For CatBoostClassifier')
plt.legend()
plt.show()

In [ ]:
model = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(6, 4), random_state=1)
train_sizes, train_scores, test_scores = learning_curve(model, market_data.loc[:,predictors],market_data.loc[:,"target"], cv=10)

plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel('Training Size')
plt.ylabel('Score')
plt.title('Learning Curve For Small Neural Network')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))

# Plot each column separately
for column in error_ma_crossover.columns:
    plt.plot(error_ma_crossover.index, error_ma_crossover[column], label=column)

plt.title('Cross Validated Moving Average Crossover Accuracy')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend(title='Models')
plt.show()

Feature Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
backward_feature_selector = SequentialFeatureSelector(LinearDiscriminantAnalysis(),
                                                      k_features=(1,market_data.loc[:,predictors].shape[1]),
                                                      forward=False,
                                                      verbose=2,
                                                      scoring="accuracy",
                                                      cv=5,
                                                      n_jobs=-1
                                                     ).fit(market_data.loc[:,predictors],market_data.loc[:,"target"])

In [ ]:
backward_feature_selector.k_feature_names_

Feature Engineering

In [ ]:
#Fetch market data
market_data = pd.DataFrame(mt5.copy_rates_range("EURUSD",timeframe,date_start,date_end))
market_data["time"] = pd.to_datetime(market_data["time"],unit='s')
#Add simple moving average technical indicator
market_data.ta.sma(length=5,append=True)
#Add simple moving average technical indicator
market_data.ta.sma(length=50,append=True)
#Add macd
market_data.ta.macd(append=True)
#Add awesome oscilator
market_data.ta.ao(append=True)
#Add aroon
market_data.ta.aroon(append=True)
#Add chaikins comodity index
market_data.ta.cci(append=True)
#Add percent return
market_data.ta.percent_return(append=True)
#Delete missing rows
market_data.dropna(inplace=True)
#Add the target
market_data["target"] = 0
market_data.loc[market_data["SMA_5"].shift(-look_ahead) > market_data["SMA_50"].shift(-look_ahead),"target"] = 1
market_data.loc[market_data["SMA_5"].shift(-look_ahead) < market_data["SMA_50"].shift(-look_ahead),"target"] = 0
#The last rows do not have answers
market_data = market_data[:-look_ahead]
market_data

In [ ]:
market_data.iloc[:,1:-1]

In [ ]:
backward_feature_selector = SequentialFeatureSelector(LinearDiscriminantAnalysis(),
                                                      k_features=(1,market_data.loc[:,predictors].shape[1]),
                                                      forward=False,
                                                      verbose=2,
                                                      scoring="accuracy",
                                                      cv=5,
                                                      n_jobs=-1
                                                     ).fit(market_data.iloc[:,1:-1],market_data.loc[:,"target"])

In [ ]:
backward_feature_selector.k_feature_names_

Building our trading algorithm

In [ ]:
#Fetch market data
market_data = pd.DataFrame(mt5.copy_rates_range("EURUSD",timeframe,date_start,date_end))
market_data["time"] = pd.to_datetime(market_data["time"],unit='s')
predictors = ['close','tick_volume','spread','SMA_5','SMA_50','MACDh_12_26_9','AO_5_34']
#Add simple moving average technical indicator
market_data.ta.sma(length=5,append=True)
#Add simple moving average technical indicator
market_data.ta.sma(length=50,append=True)
#Add macd
market_data.ta.macd(append=True)
#Add awesome oscilator
market_data.ta.ao(append=True)
#Add aroon
market_data.ta.aroon(append=True)
#Add chaikins comodity index
market_data.ta.cci(append=True)
#Add percent return
market_data.ta.percent_return(append=True)
#Delete missing rows
market_data.dropna(inplace=True)
#Add the target
#Encoding the target
ma_cross_conditions = [
    (market_data["SMA_5"].shift(-look_ahead) > market_data["SMA_50"].shift(-look_ahead)),
    (market_data["SMA_5"].shift(-look_ahead) < market_data["SMA_50"].shift(-look_ahead))
]
#Encoding pattern
ma_cross_choices = [
    #Fast MA above Slow MA
    1,
    #Fast MA below Slow MA
    0
]
market_data["target"] = np.select(ma_cross_conditions,ma_cross_choices)
#The last rows do not have answers
market_data = market_data[:-look_ahead]

In [ ]:
model = LinearDiscriminantAnalysis()
model.fit(market_data.loc[:,predictors],market_data.loc[:,"target"])

In [ ]:
def get_prices():
    start = datetime(2024,6,1)
    end   = datetime.now()
    data  = pd.DataFrame(mt5.copy_rates_range("EURUSD",timeframe,start,end))
    #Add simple moving average technical indicator
    data.ta.sma(length=5,append=True)
    data.ta.sma(length=50,append=True)
    #Add awesome oscilator
    data.ta.ao(append=True)
    #Add macd
    data.ta.macd(append=True)
    #Delete missing rows
    data.dropna(inplace=True)
    data['time'] = pd.to_datetime(data['time'],unit='s')
    data.set_index('time',inplace=True)
    data = data.loc[:,['close','tick_volume','spread','SMA_5','SMA_50','MACDh_12_26_9','AO_5_34']]
    data = data.iloc[-2:,:]
    return(data)

In [ ]:
#Get signals LDA model
def ai_signal(input_data,_model):
    #Get a forecast
    forecast = _model.predict(input_data)
    return forecast[1]

In [ ]:
#Now we define the main body of our Python Moving Average Crossover Trading Bot
if __name__ == '__main__':
    #We'll use an infinite loop to keep the program running
    while True:
        #Fetching model prediction
        signal = ai_signal(get_prices(),model)

        #Decoding model prediction into an action
        if signal == 1:
            direction = 'buy'
        elif signal == 0:
            direction = 'sell'

        print(f'AI Forecast: {direction}')

        #Opening A Buy Trade
        #But first we need to ensure there are no opposite trades open on the same symbol
        if direction == 'buy':
            #Close any sell positions
            for pos in mt5.positions_get():
                if pos.type == 1:
                    #This is an open sell order, and we need to close it
                    close_order(pos.ticket)

            if not mt5.positions_totoal():
                #We have no open positions
                mt5.Buy(symbol,volume)

        #Opening A Sell Trade
        elif direction == 'sell':
            #Close any buy positions
            for pos in mt5.positions_get():
                if pos.type == 0:
                    #This is an open buy order, and we need to close it
                    close_order(pos.ticket)

            if not mt5.positions_get():
                #We have no open positions
                mt5.sell(symbol,volume)

        print('time: ', datetime.now())
        print('-------\n')
        time.sleep(60)